In [1]:
import pandas as pd
import numpy as np

In [4]:
 #df = pd.read_csv('WMT.csv')
    #Uncomment this and read the WMT csv file wherever its stored on your local device

In [5]:
#This is where you will code up the technical indicators and add as columns in the original dataframe

def rsi(): #code rsi 6 to 25
    
def cmo():
    
def williams():
    
def sma(): #code sma 6 to 30 
    
def wma():
    
def ema():
    
def roc():
    
def cmfi():

def hma():
    
def dmi():
    
def tripleema():
    
def psi():
    
def cci():
    


IndentationError: expected an indented block (<ipython-input-5-cae2ff3fa008>, line 5)

In [6]:
def label_data():

SyntaxError: unexpected EOF while parsing (<ipython-input-6-c7c1a64e3c53>, line 1)